### 1-2-ScansManualImprovements.ipynb

Rename wings with incorrect OCR results, adjust left/right wing identification, and wings marked with an 'x'.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import sys
import os
import cv2

from pathlib import Path
from tqdm import tqdm
import shutil

In [2]:
# Define data directories
data_dir = Path("/mnt/g/Projects/Master/Data/")

input_dir = data_dir / "Processed" / "WingScans" / "1-ScansWingCrops" 
output_dir = data_dir / "Processed" / "WingScans" / "2-ScansManualImprovements" 

In [3]:
# Dictionary of files to rename
rename_dict = {"Hive01_Sheet_02_Label_121_35_Wing-1": "Hive01_Sheet_02_Label_121_35_Left_x",
            "Hive01_Sheet_02_Label_121_36_Wing-2": "Hive01_Sheet_02_Label_121_36_Right_x",
            "Hive01_Sheet_02_Label_w16_21_Left": "Hive01_Sheet_02_Label_ww16_21_Left",
            "Hive01_Sheet_02_Label_w16_22_Right": "Hive01_Sheet_02_Label_ww16_22_Right",
            "Hive01_Sheet_03_Label_ww49_5_Only": "Hive01_Sheet_03_Label_ww49_5_Right",
            "Hive02_Sheet_01_Label_33_21_Only": "Hive02_Sheet_01_Label_33_21_Right",
            "Hive02_Sheet_02_Label_64_35_Only": "Hive02_Sheet_02_Label_64_35_Left",
            "Hive02_Sheet_03_Label_ww19_7_Only": "Hive02_Sheet_03_Label_ww19_7_Right",
            "Hive02_Sheet_03_Label_ww35_20_Only": "Hive02_Sheet_03_Label_ww35_20_Right",
            "Hive03_Sheet_01_Label_31_18_Only": "Hive03_Sheet_01_Label_31_18_Left",
            "Hive03_Sheet_01_Label_115_23_Only": "Hive03_Sheet_01_Label_115_23_Right",
            "Hive03_Sheet_01_Label_ww33_9_Only": "Hive03_Sheet_01_Label_ww33_9_Right",
            "Hive03_Sheet_01_Label_w41_5_Left": "Hive03_Sheet_01_Label_ww41_5_Left",
            "Hive03_Sheet_01_Label_w41_6_Right": "Hive03_Sheet_01_Label_ww41_6_Right",
            "Hive03_Sheet_01_Label_ww7_16_Left": "Hive03_Sheet_01_Label_ww17_16_Left",
            "Hive03_Sheet_01_Label_ww7_17_Right": "Hive03_Sheet_01_Label_ww17_17_Right",
            "Hive05_Sheet_01_Label_118_19_Only": "Hive05_Sheet_01_Label_118_19_Right",
            "Hive05_Sheet_01_Label_w116_3_Left": "Hive05_Sheet_01_Label_ww16_3_Left",
            "Hive05_Sheet_01_Label_w116_4_Right": "Hive05_Sheet_01_Label_ww16_4_Right",
            "Hive06_Sheet_02_Label_54_22_Wing-2": "Hive06_Sheet_02_Label_54_22_Left",
            "Hive06_Sheet_02_Label_54_23_Wing-3": "Hive06_Sheet_02_Label_54_23_Right",
            "Hive06_Sheet_04_Label_127_7_Left": "Hive06_Sheet_04_Label_127_7_Left_x",
            "Hive06_Sheet_04_Label_127_8_Right": "Hive06_Sheet_04_Label_127_8_Right_x",
            "Hive06_Sheet_04_Label_129_35_Left": "Hive06_Sheet_04_Label_129_35_Left_x",
            "Hive06_Sheet_04_Label_129_36_Right": "Hive06_Sheet_04_Label_129_36_Right_x",
            "Hive06_Sheet_04_Label_135_41_Left": "Hive06_Sheet_04_Label_135_41_Left_x",
            "Hive06_Sheet_04_Label_135_42_Right": "Hive06_Sheet_04_Label_135_42_Right_x",
            "Hive06_Sheet_05_Label_129_19_Left": "Hive06_Sheet_05_Label_129_19_Left_x",
            "Hive06_Sheet_05_Label_129_20_Right": "Hive06_Sheet_05_Label_129_20_Right_x",
            "Hive06_Sheet_05_Label_135_21_Left": "Hive06_Sheet_05_Label_135_21_Left_x",
            "Hive06_Sheet_05_Label_135_22_Right": "Hive06_Sheet_05_Label_135_22_Right_x",
            "Hive06_Sheet_05_Label_127_35_Left": "Hive06_Sheet_05_Label_127_35_Left_x",
            "Hive06_Sheet_05_Label_127_36_Right": "Hive06_Sheet_05_Label_127_36_Right_x",
            "Hive07_Sheet_01_Label_42_2_Wing-2": "Hive07_Sheet_01_Label_42_2_Left",
            "Hive07_Sheet_01_Label_42_4_Wing-4": "Hive07_Sheet_01_Label_42_4_Right",
            "Hive07_Sheet_03_Label_101_26_Only": "Hive07_Sheet_03_Label_101_26_Left",
            "Hive07_Sheet_03_Label_112_17_Only": "Hive07_Sheet_03_Label_112_17_Right",
            "Hive07_Sheet_04_Label_118_24_Wing-2": "Hive07_Sheet_04_Label_118_24_Left_x",
            "Hive07_Sheet_04_Label_118_25_Wing-3": "Hive07_Sheet_04_Label_118_25_Right_x",
            "Hive07_Sheet_05_Label_119_33_Left": "Hive07_Sheet_05_Label_119_33_Left_x",
            "Hive07_Sheet_05_Label_119_34_Right": "Hive07_Sheet_05_Label_119_34_Right_x",
            "Hive07_Sheet_06_Label_118_8_Left": "Hive07_Sheet_06_Label_118_8_Left_x",
            "Hive07_Sheet_06_Label_118_9_Right": "Hive07_Sheet_06_Label_118_9_Right_x",
            "Hive08_Sheet_03_Label_XX_7_Only": "Hive08_Sheet_03_Label_7_7_Right",
            "Hive08_Sheet_04_Label_135_30_Wing-2": "Hive08_Sheet_04_Label_135_30_Left",
            "Hive08_Sheet_04_Label_135_31_Wing-3": "Hive08_Sheet_04_Label_135_31_Right",
            "Hive08_Sheet_06_Label_57_6_Wing-2": "Hive08_Sheet_06_Label_57_6_Left",
            "Hive08_Sheet_06_Label_57_7_Wing-3": "Hive08_Sheet_06_Label_57_7_Right",
            "Hive08_Sheet_06_Label_75_30_Only": "Hive08_Sheet_06_Label_75_30_Left",
            "Hive10_Sheet_01_Label_XX_7_Left": "Hive10_Sheet_01_Label_7_7_Left",
            "Hive10_Sheet_01_Label_XX_8_Right": "Hive10_Sheet_01_Label_7_8_Right",
            "Hive10_Sheet_03_Label_87_35_Only": "Hive10_Sheet_03_Label_87_35_Left",
            "Hive10_Sheet_04_Label_119_8_Wing-2": "Hive10_Sheet_04_Label_119_8_Left",
            "Hive10_Sheet_04_Label_119_9_Wing-3": "Hive10_Sheet_04_Label_119_9_Right",
            "Hive10_Sheet_05_Label_144_33_Wing-2": "Hive10_Sheet_05_Label_144_33_Left",
            "Hive10_Sheet_05_Label_144_34_Wing-3": "Hive10_Sheet_05_Label_144_34_Right",
            "Hive10_Sheet_05_Label_161_4_Wing-2": "Hive10_Sheet_05_Label_161_4_Left",
            "Hive10_Sheet_05_Label_161_5_Wing-3": "Hive10_Sheet_05_Label_161_5_Right",
            "Hive11_Sheet_01_Label_1_27_Left": "Hive11_Sheet_01_Label_7_27_Left",
            "Hive11_Sheet_01_Label_1_28_Right": "Hive11_Sheet_01_Label_7_28_Right",
            "Hive12_Sheet_01_Label_23_35_Only": "Hive12_Sheet_01_Label_23_35_Left",
            "Hive12_Sheet_01_Label_1_23_Left": "Hive12_Sheet_01_Label_7_23_Left",
            "Hive12_Sheet_01_Label_1_24_Right": "Hive12_Sheet_01_Label_7_24_Right",
            "Hive13_Sheet_02_Label_ww32_15_Only": "Hive13_Sheet_02_Label_ww32_15_Right",
            "Hive14_Sheet_01_Label_ww18_17_Only": "Hive14_Sheet_01_Label_ww18_17_Left",
            "Hive15_Sheet_02_Label_90_13_Left": "Hive40_Sheet_02_Label_90_13_Left",
            "Hive15_Sheet_02_Label_90_14_Right": "Hive40_Sheet_02_Label_90_14_Right",
            "Hive15_Sheet_02_Label_83_17_Left": "Hive40_Sheet_02_Label_83_17_Left",
            "Hive15_Sheet_02_Label_83_18_Right": "Hive40_Sheet_02_Label_83_18_Right",
            "Hive16_Sheet_01_Label_XX_47_Left": "Hive16_Sheet_01_Label_7_47_Left",
            "Hive16_Sheet_01_Label_XX_48_Right": "Hive16_Sheet_01_Label_7_48_Right",
            "Hive16_Sheet_02_Label_38_11_Only": "Hive16_Sheet_02_Label_38_11_Left",
            "Hive16_Sheet_02_Label_44_1_Left": "Hive16_Sheet_02_Label_ww44_1_Left",
            "Hive16_Sheet_02_Label_44_2_Right": "Hive16_Sheet_02_Label_ww44_2_Right",
            "Hive16_Sheet_02_Label_42_20_Left": "Hive16_Sheet_02_Label_ww42_20_Left",
            "Hive16_Sheet_02_Label_42_21_Right": "Hive16_Sheet_02_Label_ww42_21_Right",
            "Hive17_Sheet_01_Label_XX_15_Left": "Hive17_Sheet_01_Label_7_15_Left",
            "Hive17_Sheet_01_Label_XX_16_Right": "Hive17_Sheet_01_Label_7_16_Right",
            "Hive18_Sheet_01_Label_10_7_Only": "Hive18_Sheet_01_Label_10_7_Left",
            "Hive21_Sheet_01_Label_20_15_Only": "Hive21_Sheet_01_Label_20_15_Left",
            "Hive25_Sheet_01_Label_70_33_Only": "Hive25_Sheet_01_Label_70_33_Right",
            "Hive28_Sheet_02_Label_33_39_Only": "Hive28_Sheet_02_Label_33_39_Left",
            "Hive28_Sheet_03_Label_84_21_Only": "Hive28_Sheet_03_Label_84_21_Right",
            "Hive28_Sheet_03_Label_93_32_Only": "Hive28_Sheet_03_Label_93_32_Left",
            "Hive27_Sheet_01_Label_XX_15_Left": "Hive27_Sheet_01_Label_7_15_Left",
            "Hive27_Sheet_01_Label_XX_16_Right": "Hive27_Sheet_01_Label_7_16_Right",
            "Hive30_Sheet_01_Label_XX_15_Left": "Hive30_Sheet_01_Label_7_15_Left",
            "Hive30_Sheet_01_Label_XX_16_Right": "Hive30_Sheet_01_Label_7_16_Right",
            "Hive38_Sheet_02_Label_44_7_Only": "Hive38_Sheet_02_Label_44_7_Right",
            "Hive39_Sheet_03_Label_104_17_Only": "Hive39_Sheet_03_Label_104_17_Left", 
            "Hive39_Sheet_01_Label_22_34_Wing-2": "Hive39_Sheet_01_Label_22_34_Left",
            "Hive39_Sheet_01_Label_22_35_Wing-3": "Hive39_Sheet_01_Label_22_35_Right",
            "Hive40_Sheet_04_Label_127_6_Wing-2": "Hive40_Sheet_04_Label_127_6_Left",
            "Hive40_Sheet_04_Label_127_7_Wing-3": "Hive40_Sheet_04_Label_127_7_Right",
            "HiveC5_Sheet_02_Label_25_74_Wing-2": "HiveC5_Sheet_02_Label_25_74_Left",
            "HiveC5_Sheet_02_Label_25_75_Wing-3": "HiveC5_Sheet_02_Label_25_75_Right",
            "HiveC5_Sheet_02_Label_26_59_Wing-2": "HiveC5_Sheet_02_Label_26_59_Left",
            "HiveC5_Sheet_02_Label_26_61_Wing-4": "HiveC5_Sheet_02_Label_26_61_Right",
            "HiveC5_Sheet_02_Label_27_43_Wing-2": "HiveC5_Sheet_02_Label_27_43_Left",
            "HiveC5_Sheet_02_Label_27_45_Wing-4": "HiveC5_Sheet_02_Label_27_45_Right",
            "HiveC5_Sheet_02_Label_28_27_Wing-2": "HiveC5_Sheet_02_Label_28_27_Left",
            "HiveC5_Sheet_02_Label_28_29_Wing-4": "HiveC5_Sheet_02_Label_28_29_Right",
            "HiveC5_Sheet_02_Label_29_12_Wing-2": "HiveC5_Sheet_02_Label_29_12_Left",
            "HiveC5_Sheet_02_Label_29_13_Wing-3": "HiveC5_Sheet_02_Label_29_13_Right",
            "HiveC5_Sheet_02_Label_31_77_Wing-2": "HiveC5_Sheet_02_Label_31_77_Left",
            "HiveC5_Sheet_02_Label_31_78_Wing-3": "HiveC5_Sheet_02_Label_31_78_Right",
            "HiveC5_Sheet_02_Label_32_63_Wing-2": "HiveC5_Sheet_02_Label_32_63_Left",
            "HiveC5_Sheet_02_Label_32_64_Wing-3": "HiveC5_Sheet_02_Label_32_64_Right",
            "HiveC5_Sheet_02_Label_33_47_Wing-2": "HiveC5_Sheet_02_Label_33_47_Left",
            "HiveC5_Sheet_02_Label_33_49_Wing-4": "HiveC5_Sheet_02_Label_33_49_Right",
            "HiveC5_Sheet_02_Label_34_31_Wing-2": "HiveC5_Sheet_02_Label_34_31_Left",
            "HiveC5_Sheet_02_Label_34_33_Wing-4": "HiveC5_Sheet_02_Label_34_33_Right",
            "HiveC5_Sheet_02_Label_35_15_Wing-2": "HiveC5_Sheet_02_Label_35_15_Left",
            "HiveC5_Sheet_02_Label_35_17_Wing-4": "HiveC5_Sheet_02_Label_35_17_Right",
            "HiveC5_Sheet_02_Label_36_2_Wing-2": "HiveC5_Sheet_02_Label_36_2_Left",
            "HiveC5_Sheet_02_Label_36_3_Wing-3": "HiveC5_Sheet_02_Label_36_3_Right",
            "HiveC5_Sheet_02_Label_37_80_Wing-2": "HiveC5_Sheet_02_Label_37_80_Left",
            "HiveC5_Sheet_02_Label_37_82_Wing-4": "HiveC5_Sheet_02_Label_37_82_Right",
            "HiveC5_Sheet_02_Label_38_66_Wing-2": "HiveC5_Sheet_02_Label_38_66_Left",
            "HiveC5_Sheet_02_Label_38_68_Wing-4": "HiveC5_Sheet_02_Label_38_68_Right",
            "HiveC5_Sheet_02_Label_39_51_Wing-2": "HiveC5_Sheet_02_Label_39_51_Left",
            "HiveC5_Sheet_02_Label_39_53_Wing-4": "HiveC5_Sheet_02_Label_39_53_Right",
            "HiveC5_Sheet_02_Label_40_35_Wing-2": "HiveC5_Sheet_02_Label_40_35_Left",
            "HiveC5_Sheet_02_Label_40_37_Wing-4": "HiveC5_Sheet_02_Label_40_37_Right",
            "HiveC5_Sheet_02_Label_41_19_Wing-2": "HiveC5_Sheet_02_Label_41_19_Left",
            "HiveC5_Sheet_02_Label_41_21_Wing-4": "HiveC5_Sheet_02_Label_41_21_Right",
            "HiveC5_Sheet_02_Label_42_5_Wing-2": "HiveC5_Sheet_02_Label_42_5_Left",
            "HiveC5_Sheet_02_Label_42_6_Wing-3": "HiveC5_Sheet_02_Label_42_6_Right",
            "HiveC5_Sheet_02_Label_43_84_Wing-2": "HiveC5_Sheet_02_Label_43_84_Left",
            "HiveC5_Sheet_02_Label_43_86_Wing-4": "HiveC5_Sheet_02_Label_43_86_Right",
            "HiveC5_Sheet_02_Label_44_70_Wing-2": "HiveC5_Sheet_02_Label_44_70_Left",
            "HiveC5_Sheet_02_Label_44_72_Wing-4": "HiveC5_Sheet_02_Label_44_72_Right",
            "HiveC5_Sheet_02_Label_45_55_Wing-2": "HiveC5_Sheet_02_Label_45_55_Left",
            "HiveC5_Sheet_02_Label_45_57_Wing-4": "HiveC5_Sheet_02_Label_45_57_Right",
            "HiveC5_Sheet_02_Label_46_39_Wing-2": "HiveC5_Sheet_02_Label_46_39_Left",
            "HiveC5_Sheet_02_Label_46_41_Wing-4": "HiveC5_Sheet_02_Label_46_41_Right",
            "HiveC5_Sheet_02_Label_47_23_Wing-2": "HiveC5_Sheet_02_Label_47_23_Left",
            "HiveC5_Sheet_02_Label_47_25_Wing-4": "HiveC5_Sheet_02_Label_47_25_Right",
            "HiveC5_Sheet_02_Label_48_8_Wing-2": "HiveC5_Sheet_02_Label_48_8_Left",
            "HiveC5_Sheet_02_Label_48_10_Wing-4": "HiveC5_Sheet_02_Label_48_10_Right",
            "Hive01_Sheet_03_Label_ww21_2_Right": "Hive01_Sheet_03_Label_ww21_2_Right_x",
            "Hive07_Sheet_04_Label_119_15_Left": "Hive07_Sheet_04_Label_119_15_Left_x",
            "Hive07_Sheet_04_Label_119_15_Right": "Hive07_Sheet_04_Label_119_15_Right_x"
            }

# List of files to remove with text and markings that are missidentified as wings 
markings = ["HiveC5_Sheet_02_Label_26_58_Wing-1",
            "HiveC5_Sheet_02_Label_26_60_Wing-3",
            "HiveC5_Sheet_02_Label_27_42_Wing-1",
            "HiveC5_Sheet_02_Label_27_44_Wing-3",
            "HiveC5_Sheet_02_Label_28_26_Wing-1",
            "HiveC5_Sheet_02_Label_28_28_Wing-3",
            "HiveC5_Sheet_02_Label_29_11_Wing-1",
            "HiveC5_Sheet_02_Label_31_76_Wing-1",
            "HiveC5_Sheet_02_Label_32_62_Wing-1",
            "HiveC5_Sheet_02_Label_33_46_Wing-1",
            "HiveC5_Sheet_02_Label_33_48_Wing-3",
            "HiveC5_Sheet_02_Label_34_30_Wing-1",
            "HiveC5_Sheet_02_Label_34_32_Wing-3",
            "HiveC5_Sheet_02_Label_35_14_Wing-1",
            "HiveC5_Sheet_02_Label_35_16_Wing-3",
            "HiveC5_Sheet_02_Label_36_1_Wing-1",
            "HiveC5_Sheet_02_Label_37_79_Wing-1",
            "HiveC5_Sheet_02_Label_37_81_Wing-3",
            "HiveC5_Sheet_02_Label_38_65_Wing-1",
            "HiveC5_Sheet_02_Label_38_67_Wing-3",
            "HiveC5_Sheet_02_Label_39_50_Wing-1",
            "HiveC5_Sheet_02_Label_39_52_Wing-3",
            "HiveC5_Sheet_02_Label_40_34_Wing-1",
            "HiveC5_Sheet_02_Label_40_36_Wing-3",
            "HiveC5_Sheet_02_Label_41_18_Wing-1",
            "HiveC5_Sheet_02_Label_41_20_Wing-3",
            "HiveC5_Sheet_02_Label_42_4_Wing-1",
            "HiveC5_Sheet_02_Label_43_83_Wing-1",
            "HiveC5_Sheet_02_Label_43_85_Wing-3",
            "HiveC5_Sheet_02_Label_44_69_Wing-1",
            "HiveC5_Sheet_02_Label_44_71_Wing-3",
            "HiveC5_Sheet_02_Label_45_54_Wing-1",
            "HiveC5_Sheet_02_Label_45_56_Wing-3",
            "HiveC5_Sheet_02_Label_46_38_Wing-1",
            "HiveC5_Sheet_02_Label_46_40_Wing-3",
            "HiveC5_Sheet_02_Label_47_22_Wing-1",
            "HiveC5_Sheet_02_Label_47_24_Wing-3",
            "HiveC5_Sheet_02_Label_48_7_Wing-1",
            "HiveC5_Sheet_02_Label_48_9_Wing-3",
            "HiveC5_Sheet_02_Label_25_73_Wing-1",
            "Hive05_Sheet_01_Label_ww92_20_Only",
            "Hive01_Sheet_03_Label_ww21_1_Left",
            "Hive01_Sheet_02_Label_121_37_Wing-3",
            "Hive07_Sheet_06_Label_0_1_Only",
            "Hive29_Sheet_03_Label_90_41_Only",
            "Hive38_Sheet_03_Label_0_1_Left",
            "Hive28_Sheet_01_Label_17_5_Only",
            "Hive38_Sheet_03_Label_0_2_Right",
            "Hive40_Sheet_03_Label_87_30_Right",
            "Hive06_Sheet_02_Label_54_21_Wing-1",
            "Hive06_Sheet_02_Label_54_24_Wing-4",
            "Hive07_Sheet_01_Label_42_1_Wing-1",
            "Hive07_Sheet_01_Label_42_3_Wing-3",
            "Hive07_Sheet_04_Label_118_23_Wing-1",
            "Hive08_Sheet_04_Label_135_29_Wing-1",
            "Hive08_Sheet_06_Label_57_5_Wing-1",
            "Hive10_Sheet_04_Label_119_7_Wing-1",
            "Hive10_Sheet_05_Label_144_32_Wing-1",
            "Hive39_Sheet_01_Label_22_33_Wing-1",
            "Hive10_Sheet_05_Label_161_3_Wing-1",
            "Hive40_Sheet_04_Label_127_5_Wing-1"]

In [4]:
DEBUG = False

# Ensure the input directory exists
if not os.path.exists(input_dir):
    raise FileNotFoundError(f"Input directory was not found.")

# Create the output directory
if os.path.exists(output_dir):
    raise FileExistsError(f"Output directory already exists.") 
os.makedirs(output_dir)

# Loop through all directories
all_directories = [entry for entry in input_dir.iterdir() if entry.is_dir()]
for input_subdir in tqdm(all_directories, desc="Processing files"):
    if "Hive" not in input_subdir.name:
        continue
        
    # Find jpg files
    jpg_files = list(input_subdir.glob("*.jpg"))
    for jpg_file in jpg_files:
        filename = jpg_file.stem
        
        # Skip files in the markings list
        if filename in markings:
            continue
                
        # Rename if in rename_dict, otherwise keep original name
        new_filename = rename_dict.get(filename, filename) + ".jpg"
    
        # Define full output path
        output_path = output_dir / new_filename
    
        # Copy file
        shutil.copy(jpg_file, output_path)  

Processing files: 100%|███████████████████████████████████████████████████████████████| 109/109 [01:00<00:00,  1.81it/s]
